In [85]:
from sqlalchemy import create_engine, text
import pandas as pd
from ml_experiments.analyze import get_df_runs_from_mlflow_sql, get_missing_entries, get_common_combinations, get_df_with_combinations
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from pathlib import Path
import os
import pickle
from functools import partial

# Save Results

## Load mlflow runs

In [86]:
db_port = 5001
db_name = 'cohirf'
url = f'postgresql://belucci@localhost:{db_port}/{db_name}'
engine = create_engine(url)
query = 'SELECT experiments.name from experiments'
experiment_names = pd.read_sql(query, engine)['name'].tolist()
results_dir = Path.cwd().parent / "results" / "real"
os.makedirs(results_dir, exist_ok=True)

In [87]:
experiment_names

['Default',
 'sfni-SpectralSubspaceRandomization',
 'sfni-KMeans',
 'sfni-BatchCoHiRF-SC-SRGF',
 'sfni-BatchCoHiRF-1iter',
 'sfni-CoHiRF',
 'sphere-BatchCoHiRF-DBSCAN-1iter',
 'sphere-DBSCAN',
 'sphere-CoHiRF-DBSCAN',
 'real-ari-BatchCoHiRF-1iter',
 'real-ari-AverageAgglomerativeClustering',
 'real-ari-BatchCoHiRF-DBSCAN-1iter',
 'real-ari-AffinityPropagation',
 'real-ari-BatchCoHiRF-SC-SRGF',
 'real-ari-CoHiRF-DBSCAN',
 'real-ari-CoHiRF-KernelRBF',
 'real-ari-CoHiRF',
 'real-ari-CompleteAgglomerativeClustering',
 'real-ari-DBSCAN',
 'real-ari-HDBSCAN',
 'real-ari-IRFLLRR',
 'real-ari-KMeans',
 'real-ari-MeanShift',
 'real-ari-OPTICS',
 'real-ari-Proclus',
 'real-ari-SingleAgglomerativeClustering',
 'real-ari-SpectralClustering',
 'real-ari-SpectralSubspaceRandomization',
 'real-ari-WardAgglomerativeClustering',
 'real-adjusted_mutual_info-BatchCoHiRF-DBSCAN-1iter',
 'real-adjusted_mutual_info-DBSCAN',
 'real-adjusted_mutual_info-BatchCoHiRF-SC-SRGF',
 'real-adjusted_mutual_info-Comple

In [88]:
experiments_names = [exp for exp in experiment_names if exp.startswith("real-")]

In [89]:
experiments_names

['real-ari-BatchCoHiRF-1iter',
 'real-ari-AverageAgglomerativeClustering',
 'real-ari-BatchCoHiRF-DBSCAN-1iter',
 'real-ari-AffinityPropagation',
 'real-ari-BatchCoHiRF-SC-SRGF',
 'real-ari-CoHiRF-DBSCAN',
 'real-ari-CoHiRF-KernelRBF',
 'real-ari-CoHiRF',
 'real-ari-CompleteAgglomerativeClustering',
 'real-ari-DBSCAN',
 'real-ari-HDBSCAN',
 'real-ari-IRFLLRR',
 'real-ari-KMeans',
 'real-ari-MeanShift',
 'real-ari-OPTICS',
 'real-ari-Proclus',
 'real-ari-SingleAgglomerativeClustering',
 'real-ari-SpectralClustering',
 'real-ari-SpectralSubspaceRandomization',
 'real-ari-WardAgglomerativeClustering',
 'real-adjusted_mutual_info-BatchCoHiRF-DBSCAN-1iter',
 'real-adjusted_mutual_info-DBSCAN',
 'real-adjusted_mutual_info-BatchCoHiRF-SC-SRGF',
 'real-adjusted_mutual_info-CompleteAgglomerativeClustering',
 'real-adjusted_mutual_info-AverageAgglomerativeClustering',
 'real-adjusted_mutual_info-BatchCoHiRF-1iter',
 'real-adjusted_mutual_info-CoHiRF',
 'real-adjusted_mutual_info-CoHiRF-KernelRBF

In [90]:
query = "SELECT DISTINCT(key) FROM params WHERE key LIKE 'best/%%'"
best_params = pd.read_sql(query, engine)["key"].tolist()

In [91]:
params_columns = [
    "model",
    "dataset_id",
	"n_trials",
	"dataset_name",
	"standardize",
	"hpo_metric",
] + best_params

In [92]:
latest_metrics_columns = [
    "fit_model_return_elapsed_time",
    "max_memory_used_after_fit",
    "max_memory_used",
	"best/n_clusters_",
    "best/rand_score",
    "best/adjusted_rand",
    "best/mutual_info",
    "best/adjusted_mutual_info",
    "best/normalized_mutual_info",
    "best/homogeneity_completeness_v_measure",
    "best/silhouette",
    "best/calinski_harabasz_score",
    "best/davies_bouldin_score",
    "best/inertia_score",
    "best/homogeneity",
    "best/completeness",
    "best/v_measure",
    "best/elapsed_time",
]

In [93]:
tags_columns = [
    'raised_exception',
    'EXCEPTION',
    'mlflow.parentRunId',
]

In [94]:
runs_columns = ['run_uuid', 'status', 'start_time', 'end_time']
experiments_columns = []
other_table = 'params'
other_table_keys = params_columns
df_params = get_df_runs_from_mlflow_sql(engine, runs_columns=runs_columns, experiments_columns=experiments_columns, experiments_names=experiments_names, other_table=other_table, other_table_keys=other_table_keys)
df_latest_metrics = get_df_runs_from_mlflow_sql(engine, runs_columns=['run_uuid'], experiments_columns=experiments_columns, experiments_names=experiments_names, other_table='latest_metrics', other_table_keys=latest_metrics_columns)
df_tags = get_df_runs_from_mlflow_sql(engine, runs_columns=['run_uuid'], experiments_columns=experiments_columns, experiments_names=experiments_names, other_table='tags', other_table_keys=tags_columns)

In [95]:
dataset_characteristics = pd.read_csv(results_dir / "datasets_characteristics.csv", index_col=0)
dataset_characteristics.index = dataset_characteristics["openml_id"].astype(str)

In [96]:
df_runs_raw = df_params.join(df_latest_metrics)
df_runs_raw = df_runs_raw.join(df_tags)
df_runs_raw = df_runs_raw.join(dataset_characteristics, on='dataset_id', rsuffix='_dataset')
df_runs_raw.to_csv(results_dir / 'df_runs_raw.csv', index=True)

In [97]:
df_runs_raw = pd.read_csv(results_dir / "df_runs_raw.csv", index_col=0)
# mask = df_runs_raw["model"].str.contains("CoHiRF")
# df_runs_raw.loc[mask, "model"] = df_runs_raw.loc[mask].apply(lambda row: f"{row['model']}-{row['n_trials']}", axis=1)
df_runs_raw_parents = df_runs_raw.copy()
df_runs_raw_parents = df_runs_raw_parents.loc[df_runs_raw_parents["mlflow.parentRunId"].isna()]

In [98]:
df_runs_raw_parents

,status,start_time,end_time,best/alpha,best/avg_dims,best/base_model_kwargs/eps,best/base_model_kwargs/min_samples,best/base_model_kwargs/n_clusters,best/c,best/child_run_id,...,max_memory_used_after_fit,EXCEPTION,mlflow.parentRunId,raised_exception,dataset,openml_id,n_instances,n_features,n_classes,n_categorical
run_uuid,,,,,,,,,,,,,,,,,,,,,
0023396bbc3842289bd8a53f76ffb713,FINISHED,1751654973207,1.751655e+12,NaN,NaN,NaN,NaN,NaN,NaN,6b60e37cc36a4283afd549ac9d61c0ac,...,438.280,NaN,NaN,False,armstrong-2002-v1,46775.0,72.0,1082.0,2.0,0.0
0029b8e4dd664b4ab4274dea73d7706c,FINISHED,1751923288529,1.751923e+12,NaN,NaN,NaN,NaN,NaN,NaN,638644c225cc41a399f6a28798f58695,...,442.428,NaN,NaN,False,armstrong-2002-v1,46775.0,72.0,1082.0,2.0,0.0
002d76fd45ef426db8ad19091bc5b622,FINISHED,1751923103527,1.751995e+12,4.0,NaN,NaN,NaN,NaN,0.001,cbd70cd84cb5464286c4b5abc880a7e1,...,8327.696,NaN,NaN,False,nursery,1568.0,12958.0,9.0,4.0,9.0
003bc124a9bd4ff99c4e646bb808b509,FAILED,1752647956586,1.752648e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,428.496,Best metric normalized_mutual_info not found i...,NaN,True,armstrong-2002-v1,46775.0,72.0,1082.0,2.0,0.0
00456807551642fbbbf4793923ad310a,FAILED,1752652904069,1.752653e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,388.876,Best metric normalized_mutual_info not found i...,NaN,True,chowdary-2006,46778.0,104.0,183.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffa339c852c346fdbb15f93175d2833a,FAILED,1752927611781,1.752928e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,'PseudoKernelClustering' object has no attribu...,NaN,True,segment,40984.0,2310.0,20.0,7.0,1.0
ffc40bd60f984c58b99771490b251d00,FINISHED,1751930924269,1.751931e+12,NaN,NaN,NaN,NaN,NaN,NaN,b717c58598a54fb5a806497b432ec9f2,...,419.780,NaN,NaN,False,satimage,182.0,6430.0,37.0,6.0,1.0
ffdf87efd29e444a92883d4f4292934e,FAILED,1752591826177,1.752592e+12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,621.404,Best metric silhouette not found in the best t...,NaN,True,garber-2001,46779.0,66.0,4554.0,4.0,0.0


## Delete duplicate runs (if any) and complete some models that cannot run with some datasets

In [99]:
non_duplicate_columns = [
    "model",
    "dataset_id",
	"standardize",
	"hpo_metric",	
]
# df_runs_parents.loc[df_runs_parents["best/n_clusters_"]*0.5 > df_runs_parents["n_instances"], "best/adjusted_rand"] = 
df_runs_parents = df_runs_raw_parents.dropna(axis=0, how="all", subset=["best/adjusted_rand"]).copy()
# add back runs that were not evaluated because we judged too many clusters (but they run anyway)
# df_valid_runs = df_runs_raw_parents.loc[df_runs_raw_parents["best/n_clusters_"] > df_runs_raw_parents["n_instances"]*0.5].copy()
# df_runs_parents = pd.concat([df_runs_parents, df_valid_runs], axis=0)
df_runs_parents = df_runs_parents.loc[(~df_runs_parents.duplicated(non_duplicate_columns))]
# fill missing values with "None"
df_runs_parents = df_runs_parents.fillna("None")

In [100]:
df_to_cat = []
hpo_metrics = [
    "adjusted_rand",
    "adjusted_mutual_info",
    "calinski_harabasz_score",
    "silhouette",
    "davies_bouldin_score",
    "normalized_mutual_info",
]
standardize = [True]
fill_value = pd.NA
fill_columns = ["best/adjusted_rand", "best/adjusted_mutual_info", "best/calinski_harabasz_score", "best/silhouette", "best/davies_bouldin_score", "best/normalized_mutual_info"]

In [101]:
# Too few examples (<100) to run in batch
dataset_ids_to_complete = [46773, 46774, 46775, 46776, 46777, 46779, 46780, 46781]
model_names = [
    "BatchCoHiRF-1iter",
    "BatchCoHiRF-DBSCAN-1iter",
    "BatchCoHiRF-SC-SRGF",
	'BatchCoHiRF-KernelRBF-1iter',
	"CoHiRF-DBSCAN",  # just not adapted to those datasets
]
for dataset_id in dataset_ids_to_complete:
	for model_name in model_names:
		for hpo_metric in hpo_metrics:
			for std in standardize:
				mask = (df_runs_parents["dataset_id"] == dataset_id) & (df_runs_parents["model"] == model_name) & (df_runs_parents["hpo_metric"] == hpo_metric) & (df_runs_parents["standardize"] == std)
				if not mask.any():
					new_row = {
						"dataset_id": dataset_id,
						"model": model_name,
						"hpo_metric": hpo_metric,
						"standardize": std,
					}
					for col in fill_columns:
						new_row[col] = fill_value
					df_to_cat.append(new_row)

In [102]:
# Too many examples for IRFLLRR
dataset_ids_to_complete = [40685]
model_names = [
    "IRFLLRR"
]
hpo_metrics = ["adjusted_rand", "adjusted_mutual_info", "calinski_harabasz_score", "silhouette", "davies_bouldin_score", "normalized_mutual_info"]
standardize = [True]
fill_value = pd.NA
fill_columns = ["best/adjusted_rand", "best/adjusted_mutual_info", "best/calinski_harabasz_score", "best/silhouette", "best/davies_bouldin_score", "best/normalized_mutual_info"]
for dataset_id in dataset_ids_to_complete:
    for model_name in model_names:
        for hpo_metric in hpo_metrics:
            for std in standardize:
                mask = (
                    (df_runs_parents["dataset_id"] == dataset_id)
                    & (df_runs_parents["model"] == model_name)
                    & (df_runs_parents["hpo_metric"] == hpo_metric)
                    & (df_runs_parents["standardize"] == std)
                )
                if not mask.any():
                    new_row = {
                        "dataset_id": dataset_id,
                        "model": model_name,
                        "hpo_metric": hpo_metric,
                        "standardize": std,
                    }
                    for col in fill_columns:
                        new_row[col] = fill_value
                    df_to_cat.append(new_row)

In [103]:
df_runs_parents = pd.concat([df_runs_parents, pd.DataFrame(df_to_cat)], axis=0)

# Slow / Memory intensive datasets/models

In [104]:
df = df_runs_parents.copy()
df = df[["model", "dataset_id", "max_memory_used", "max_memory_used_after_fit"]]
df = df.sort_values(by="max_memory_used", ascending=False)
high_mem_tuples = df.loc[df["max_memory_used"] > 10000].copy()
high_mem_tuples = high_mem_tuples[["model", "dataset_id"]]

In [105]:
df

,model,dataset_id,max_memory_used,max_memory_used_after_fit
ba156669bdb04a24b3e2e114390be16c,SpectralSubspaceRandomization,1568,128347.652,128347.652
10a9397979ce41f0b09d2760a7495073,SpectralSubspaceRandomization,1568,126232.556,126232.556
b0ebd933c5c24b78b26f0ba1855e44f5,AffinityPropagation,40685,123946.660,123946.660
8ebbc6b4c4d04d59a890d8e986fe11c8,AffinityPropagation,40685,123906.468,123906.468
483462f877fb4e3eb9619fcb9436bffb,AffinityPropagation,40685,123591.856,123591.856
...,...,...,...,...
225,IRFLLRR,40685,NaN,NaN
226,IRFLLRR,40685,NaN,NaN
227,IRFLLRR,40685,NaN,NaN
228,IRFLLRR,40685,NaN,NaN


# Missing

In [106]:
model_nickname = df_runs_parents['model'].unique().tolist()
model_nickname.sort()
model_nickname

['AffinityPropagation',
 'AverageAgglomerativeClustering',
 'BatchCoHiRF-1iter',
 'BatchCoHiRF-DBSCAN-1iter',
 'BatchCoHiRF-KernelRBF-1iter',
 'BatchCoHiRF-SC-SRGF',
 'CoHiRF',
 'CoHiRF-DBSCAN',
 'CoHiRF-KernelRBF',
 'CompleteAgglomerativeClustering',
 'DBSCAN',
 'HDBSCAN',
 'IRFLLRR',
 'KMeans',
 'MeanShift',
 'OPTICS',
 'Proclus',
 'SingleAgglomerativeClustering',
 'SpectralClustering',
 'SpectralSubspaceRandomization',
 'WardAgglomerativeClustering']

In [111]:
model_nickname = [
    # "AffinityPropagation",
    # "AverageAgglomerativeClustering",
    # "BatchCoHiRF",
    "BatchCoHiRF-1iter",
    # "BatchCoHiRF-DBSCAN",
    "BatchCoHiRF-DBSCAN-1iter",
    "BatchCoHiRF-KernelRBF-1iter",
    "BatchCoHiRF-SC-SRGF",
    # "Clique",
    "CoHiRF",
    "CoHiRF-DBSCAN",
    "CoHiRF-KernelRBF",
    # "CompleteAgglomerativeClustering",
    "DBSCAN",
    # "HDBSCAN",
    # "IRFLLRR",
    "KMeans",
    # "KMeansProj",
    "KernelRBFKMeans",
    # "MeanShift",
    # "OPTICS",
    # "Proclus",
    # "SingleAgglomerativeClustering",
    # "SpectralClustering",
    "SpectralSubspaceRandomization",
    # "WardAgglomerativeClustering",
]

In [112]:
non_duplicate_columns = [
	"model",
	"dataset_id",
	"standardize",
	"hpo_metric",
]

In [113]:
dataset_id = [
    39,
    61,
    182,
    1478,
    1568,
    40685,
    40984,
    46773,
    46774,
    46775,
    46776,
    46777,
    46778,
    46779,
    46780,
    46781,
    46782,
    46783,
	554,
]
standardize = [True]
hpo_metric = ["adjusted_rand"]
columns_names = non_duplicate_columns
should_contain_values = [model_nickname, dataset_id, standardize, hpo_metric]
df_missing_ari = get_missing_entries(df_runs_parents, columns_names, should_contain_values)

In [114]:
df_missing_ari

,model,dataset_id,standardize,hpo_metric
0,BatchCoHiRF-KernelRBF-1iter,39,True,adjusted_rand
1,BatchCoHiRF-KernelRBF-1iter,61,True,adjusted_rand
2,BatchCoHiRF-KernelRBF-1iter,182,True,adjusted_rand
3,BatchCoHiRF-KernelRBF-1iter,1478,True,adjusted_rand
4,BatchCoHiRF-KernelRBF-1iter,1568,True,adjusted_rand
5,BatchCoHiRF-KernelRBF-1iter,40984,True,adjusted_rand
6,BatchCoHiRF-KernelRBF-1iter,46778,True,adjusted_rand
7,BatchCoHiRF-KernelRBF-1iter,46782,True,adjusted_rand
8,BatchCoHiRF-KernelRBF-1iter,46783,True,adjusted_rand
9,BatchCoHiRF-KernelRBF-1iter,554,True,adjusted_rand


In [64]:
missing_ari_tuples = df_missing_ari[["model", "dataset_id"]].copy()

In [115]:
dataset_id = [
    39,
    61,
    182,
    1478,
    1568,
    40685,
    40984,
    46773,
    46774,
    46775,
    46776,
    46777,
    46778,
    46779,
    46780,
    46781,
    46782,
    46783,
	554,
]
standardize = [True]
hpo_metric = [
    "normalized_mutual_info",
    "adjusted_mutual_info",
    "calinski_harabasz_score",
    "silhouette",
    "davies_bouldin_score",
]
columns_names = non_duplicate_columns
should_contain_values = [model_nickname, dataset_id, standardize, hpo_metric]
df_missing_others = get_missing_entries(df_runs_parents, columns_names, should_contain_values)

In [116]:
df_missing_others

,model,dataset_id,standardize,hpo_metric
0,BatchCoHiRF-KernelRBF-1iter,39,True,normalized_mutual_info
1,BatchCoHiRF-KernelRBF-1iter,39,True,adjusted_mutual_info
2,BatchCoHiRF-KernelRBF-1iter,39,True,calinski_harabasz_score
3,BatchCoHiRF-KernelRBF-1iter,39,True,silhouette
4,BatchCoHiRF-KernelRBF-1iter,39,True,davies_bouldin_score
...,...,...,...,...
186,SpectralSubspaceRandomization,554,True,normalized_mutual_info
187,SpectralSubspaceRandomization,554,True,adjusted_mutual_info
188,SpectralSubspaceRandomization,554,True,calinski_harabasz_score
189,SpectralSubspaceRandomization,554,True,silhouette


In [117]:
# Join df_runs_raw_parents into df_missing using non_duplicate_columns to get the EXCEPTION column
df_missing_with_exception = df_missing_others.merge(
    df_runs_raw_parents[non_duplicate_columns + ["raised_exception"]],
    how="left",
    left_on=["model", "dataset_id", "standardize", "hpo_metric"],
    right_on=["model", "dataset_id", "standardize", "hpo_metric"],
)
df_missing_with_exception[["model", "dataset_id", "standardize", "hpo_metric", "raised_exception"]]

,model,dataset_id,standardize,hpo_metric,raised_exception
0,BatchCoHiRF-KernelRBF-1iter,39,True,normalized_mutual_info,True
1,BatchCoHiRF-KernelRBF-1iter,39,True,adjusted_mutual_info,True
2,BatchCoHiRF-KernelRBF-1iter,39,True,calinski_harabasz_score,True
3,BatchCoHiRF-KernelRBF-1iter,39,True,silhouette,NaN
4,BatchCoHiRF-KernelRBF-1iter,39,True,davies_bouldin_score,True
...,...,...,...,...,...
188,SpectralSubspaceRandomization,554,True,normalized_mutual_info,NaN
189,SpectralSubspaceRandomization,554,True,adjusted_mutual_info,NaN
190,SpectralSubspaceRandomization,554,True,calinski_harabasz_score,NaN
191,SpectralSubspaceRandomization,554,True,silhouette,NaN


In [118]:
df_missing_dict = df_missing_others.copy()
# get only rows from high_mem_tuples
# df_missing_dict = df_missing_dict.merge(high_mem_tuples, on=["model", "dataset_id"], how="left", indicator=True)
# df_missing_dict = df_missing_dict[df_missing_dict["_merge"] == "both"].drop(columns="_merge")
# exclude rows that are in missing_ari_tuples
df_missing_dict = df_missing_dict.merge(
	missing_ari_tuples, on=["model", "dataset_id"], how="left", indicator=True
)
df_missing_dict = df_missing_dict[df_missing_dict["_merge"] == "left_only"].drop(columns="_merge")
# exclude rows that are in high_mem_tuples
df_missing_dict = df_missing_dict.merge(
	high_mem_tuples, on=["model", "dataset_id"], how="left", indicator=True
)
df_missing_dict = df_missing_dict[df_missing_dict["_merge"] == "left_only"].drop(columns="_merge")
# to_drop = pd.concat([missing_ari_tuples, high_mem_tuples], ignore_index=True)
# df_missing_dict = df_missing_dict[df_missing_dict["_merge"] == "left_only"].drop(columns="_merge")

In [119]:
df_missing_dict

,model,dataset_id,standardize,hpo_metric
0,BatchCoHiRF-KernelRBF-1iter,39,True,normalized_mutual_info
1,BatchCoHiRF-KernelRBF-1iter,39,True,adjusted_mutual_info
2,BatchCoHiRF-KernelRBF-1iter,39,True,calinski_harabasz_score
3,BatchCoHiRF-KernelRBF-1iter,39,True,silhouette
4,BatchCoHiRF-KernelRBF-1iter,39,True,davies_bouldin_score
...,...,...,...,...
161,KernelRBFKMeans,554,True,normalized_mutual_info
162,KernelRBFKMeans,554,True,adjusted_mutual_info
163,KernelRBFKMeans,554,True,calinski_harabasz_score
164,KernelRBFKMeans,554,True,silhouette


In [120]:
missing_dict = {}
for model in df_missing_dict["model"].unique():
    sub = df_missing_dict[df_missing_dict["model"] == model].drop(columns=["model"])
    # standardize = True
    sub_standardized = sub.loc[sub["standardize"] == True].copy()
    sub_standardized["standardize"] = ''
    sub_standardized_dict = sub_standardized.to_dict(orient="records")
    # standardize = False
    sub_not_standardized = sub.loc[sub["standardize"] == False].copy()
    sub_not_standardized.drop(columns=["standardize"], inplace=True)
    sub_not_standardized_dict = sub_not_standardized.to_dict(orient="records")
    # combine both dictionaries
    missing_dict[model] = sub_standardized_dict + sub_not_standardized_dict
if len(missing_dict) != 0:
    with open(results_dir / 'missing_dict.pkl', 'wb') as f:
        pickle.dump(missing_dict, f)

In [121]:
missing_dict

{'BatchCoHiRF-KernelRBF-1iter': [{'dataset_id': 39,
   'standardize': '',
   'hpo_metric': 'normalized_mutual_info'},
  {'dataset_id': 39, 'standardize': '', 'hpo_metric': 'adjusted_mutual_info'},
  {'dataset_id': 39,
   'standardize': '',
   'hpo_metric': 'calinski_harabasz_score'},
  {'dataset_id': 39, 'standardize': '', 'hpo_metric': 'silhouette'},
  {'dataset_id': 39, 'standardize': '', 'hpo_metric': 'davies_bouldin_score'},
  {'dataset_id': 61,
   'standardize': '',
   'hpo_metric': 'normalized_mutual_info'},
  {'dataset_id': 61, 'standardize': '', 'hpo_metric': 'adjusted_mutual_info'},
  {'dataset_id': 61,
   'standardize': '',
   'hpo_metric': 'calinski_harabasz_score'},
  {'dataset_id': 61, 'standardize': '', 'hpo_metric': 'silhouette'},
  {'dataset_id': 61, 'standardize': '', 'hpo_metric': 'davies_bouldin_score'},
  {'dataset_id': 182,
   'standardize': '',
   'hpo_metric': 'normalized_mutual_info'},
  {'dataset_id': 182, 'standardize': '', 'hpo_metric': 'adjusted_mutual_info'

# Tables

In [71]:
best_params

['best/alpha',
 'best/avg_dims',
 'best/base_model_kwargs/eps',
 'best/base_model_kwargs/min_samples',
 'best/base_model_kwargs/n_clusters',
 'best/c',
 'best/child_run_id',
 'best/cohirf_kwargs/base_model_kwargs/eps',
 'best/cohirf_kwargs/base_model_kwargs/min_samples',
 'best/cohirf_kwargs/base_model_kwargs/n_similarities',
 'best/cohirf_kwargs/base_model_kwargs/sampling_ratio',
 'best/cohirf_kwargs/base_model_kwargs/sc_n_clusters',
 'best/cohirf_kwargs/kmeans_n_clusters',
 'best/cohirf_kwargs/n_features',
 'best/cohirf_kwargs/repetitions',
 'best/damping',
 'best/eps',
 'best/kmeans_n_clusters',
 'best/lambda_',
 'best/min_bin_freq',
 'best/min_cluster_size',
 'best/min_samples',
 'best/n_clusters',
 'best/n_features',
 'best/n_similarities',
 'best/p',
 'best/repetitions',
 'best/sampling_ratio',
 'best/sc_n_clusters',
 'best/transform_kwargs/gamma']

In [72]:
def get_parameters_string(row):
    parameter_names = {
		"best/alpha": "\\alpha",
		"best/avg_dims": "d",
		"best/base_model_kwargs/eps": "\\epsilon",
		"best/base_model_kwargs/min_samples": "n_{\\text{min}}",
		"best/base_model_kwargs/n_clusters": "C",
		"best/c": "c",
		"best/cohirf_kwargs/base_model_kwargs/eps": "\\epsilon",
		"best/cohirf_kwargs/base_model_kwargs/min_samples": "n_{\\text{min}}",
		"best/cohirf_kwargs/kmeans_n_clusters": "C",
		"best/cohirf_kwargs/n_features": "q",
		"best/cohirf_kwargs/repetitions": "R",
		"best/damping": "\\lambda",
		# "best/density_threshold": "\\tau",
		"best/eps": "\\epsilon",
		"best/kmeans_n_clusters": "C",
		"best/lambda_": "\\lambda",
		"best/min_bin_freq": "bin_{\\text{min}}",
		"best/min_cluster_size": "C_{\\text{min}}",
		"best/min_samples": "n_{\\text{min}}",
		"best/n_clusters": "C",
		"best/n_features": "q",
		# "best/n_partitions": "P",
		"best/n_similarities": "m",
		"best/p": "p",
		"best/repetitions": "R",
		"best/sampling_ratio": "r",
		"best/sc_n_clusters": "C",
		"best/transform_kwargs/gamma": "\\gamma",
	}
    first = True
    str = ""
    for p in parameter_names.keys():
        if not pd.isna(row[p]) and row[p] != "None":
            if not first:
                str += "; "
            else:
                first = False
            value = float(row[p])
            if value.is_integer():
                value = int(value)
                str += f"${parameter_names[p]}={value}$"
            else:
                str += f"${parameter_names[p]}={value:0.2f}$"
    return str

In [73]:
def highlight_max(df, column_name, level=0):
    df_column = df[column_name]
    max_values = df_column.groupby(level=level).transform('max')
    is_highlighted = df_column.round(3) == max_values.round(3)
    df_css = df.copy().astype(str)
    df_css.loc[:, :] = ''
    df_css[is_highlighted] = 'font-weight: bold'
    return df_css

In [74]:
def highlight_max_index(series_index, df_column, level=0):
    max_values = df_column.groupby(level=level).transform('max')
    is_highlighted = df_column.round(3) == max_values.round(3)
    series_css = series_index.copy().astype(str)
    series_css[:] = ''
    series_css[is_highlighted.values] = 'font-weight: bold'
    return series_css

In [75]:
def underline_2nd_max(df, column_name, level=0):
    df_column = df[column_name]
    # get the second max value
    second_max_values = df_column.groupby(level=level).transform(lambda x: x.round(3).drop_duplicates().nlargest(2).iloc[-1])
    is_underlined = df_column.round(3) == second_max_values.round(3)
    df_css = df.copy().astype(str)
    df_css.loc[:, :] = ''
    df_css[is_underlined] = 'underline: --latex--rwrap'
    return df_css

In [76]:
def underline_2nd_max_index(series_index, df_column, level=0):
    # get the second max value
    second_max_values = df_column.groupby(level=level).transform(lambda x: x.nlargest(2).iloc[-1])
    is_underlined = df_column.round(3) == second_max_values.round(3)
    series_css = series_index.copy().astype(str)
    series_css.loc[:] = ''
    series_css[is_underlined.values] = 'underline: --latex--rwrap'
    return series_css

## Some Models

In [77]:
df_runs_parents['dataset_name'].unique()

array(['armstrong-2002-v1', 'nursery', 'har', 'bittner-2000',
       'garber-2001', 'segment', 'chowdary-2006', 'alizadeh-2000-v2',
       'khan-2001', 'golub-1999-v2', 'iris', 'ecoli', 'bredel-2005',
       'satimage', 'binary_alpha_digits', 'coil-20', 'shuttle',
       'mnist_784', 'alizadeh-2000-v3', nan], dtype=object)

In [78]:
model_names = {
    # "AffinityPropagation": "Affinity Propagation",
    # "AverageAgglomerativeClustering": "Average Agglomerative Clustering",
    # 'BatchCoHiRF',
    "BatchCoHiRF-1iter": "Batch CoHiRF",
    # 'BatchCoHiRF-DBSCAN',
    "BatchCoHiRF-DBSCAN-1iter": "Batch CoHiRF-DBSCAN",
    'BatchCoHiRF-KernelRBF-1iter': "Batch CoHiRF-KernelRBF",
    "BatchCoHiRF-SC-SRGF": "Batch CoHiRF-SC-SRGF",
    # 'Clique',
    "CoHiRF": "CoHiRF",
    "CoHiRF-DBSCAN": "CoHiRF-DBSCAN",
    "CoHiRF-KernelRBF": "CoHiRF-KernelRBF",
    # "CompleteAgglomerativeClustering": "Complete Agglomerative Clustering",
    "DBSCAN": "DBSCAN",
    # "HDBSCAN": "HDBSCAN",
    # "IRFLLRR": "IRFLLRR",
    "KMeans": "K-Means",
    # 'KMeansProj': "K-Means Proj",
    'KernelRBFKMeans': "Kernel RBF K-Means",
    # "MeanShift": "Mean Shift",
    # "OPTICS": "OPTICS",
    # "Proclus": "Proclus",
    # "SingleAgglomerativeClustering": "Single Agglomerative Clustering",
    # "SpectralClustering": "Spectral Clustering",
    "SpectralSubspaceRandomization": "SC-SRGF",
    # "WardAgglomerativeClustering": "Ward's Method"
}

dataset_names = {
    "binary_alpha_digits": "binary-alpha-digits",
}  # otherwise we get an error in latex

# Filter to only standardized runs
df = df_runs_parents.copy()
df = df.loc[df['standardize'] == True]
df = df.loc[df['model'].isin(model_names.keys())]
df = df.replace({"model": model_names})
df = df.replace({"dataset_name": dataset_names})

# "adjusted_rand",
#     "adjusted_mutual_info",
#     "calinski_harabasz_score",
#     "silhouette",
#     "davies_bouldin_score",
#     "normalized_mutual_info",

# Create separate dataframes for each metric optimization
df_ari = df.loc[df['hpo_metric'] == 'adjusted_rand'][
	['dataset_name', 'model', 'best/adjusted_rand']
].rename(columns={'best/adjusted_rand': 'ARI'})

df_ami = df.loc[df['hpo_metric'] == 'adjusted_mutual_info'][
	['dataset_name', 'model', 'best/adjusted_mutual_info']
].rename(columns={'best/adjusted_mutual_info': 'AMI'})

df_nmi = df.loc[df["hpo_metric"] == "normalized_mutual_info"][
    ["dataset_name", "model", "best/normalized_mutual_info"]
].rename(columns={"best/normalized_mutual_info": "NMI"})

df_calinski = df.loc[df['hpo_metric'] == 'calinski_harabasz_score'][
	['dataset_name', 'model', 'best/calinski_harabasz_score']
].rename(columns={'best/calinski_harabasz_score': 'Calinski'})

df_silhouette = df.loc[df['hpo_metric'] == 'silhouette'][
    ['dataset_name', 'model', 'best/silhouette']
].rename(columns={'best/silhouette': 'Silhouette'})

df_davies_bouldin = df.loc[df['hpo_metric'] == 'davies_bouldin_score'][
    ['dataset_name', 'model', 'best/davies_bouldin_score']
].rename(columns={'best/davies_bouldin_score': 'Davies-Bouldin'})


# Remove missing values before setting index
df_ari = df_ari.dropna(subset=["ARI"])
df_ami = df_ami.dropna(subset=["AMI"])
df_nmi = df_nmi.dropna(subset=["NMI"])
df_calinski = df_calinski.dropna(subset=["Calinski"])
df_silhouette = df_silhouette.dropna(subset=["Silhouette"])
df_davies_bouldin = df_davies_bouldin.dropna(subset=["Davies-Bouldin"])

# Set multi-index for all dataframes
df_ari = df_ari.set_index(["dataset_name", "model"])
df_ami = df_ami.set_index(["dataset_name", "model"])
df_nmi = df_nmi.set_index(["dataset_name", "model"])
df_calinski = df_calinski.set_index(["dataset_name", "model"])
df_silhouette = df_silhouette.set_index(["dataset_name", "model"])
df_davies_bouldin = df_davies_bouldin.set_index(["dataset_name", "model"])

# Combine all metrics into a single dataframe using outer join to keep all combinations
df_metrics = df_ari.join(df_ami, how="outer").join(df_nmi, how="outer").join(df_calinski, how="outer").join(df_silhouette, how="outer").join(df_davies_bouldin, how="outer")

# Rename index levels
df_metrics.index.names = ["Dataset", "Model"]

In [81]:
df_metrics

ARI       AMI       NMI  \
Dataset          Model                                                
alizadeh-2000-v2 Batch CoHiRF-SC-SRGF  0.794763  0.671212       NaN   
                 CoHiRF                0.864606  0.757990  0.768753   
                 CoHiRF-KernelRBF      0.046212  0.085768  0.255128   
                 DBSCAN                0.000000  0.000000  0.000000   
                 K-Means               0.830673  0.750678  0.756502   
...                                         ...       ...       ...   
shuttle          CoHiRF                0.612660  0.499274       NaN   
                 CoHiRF-DBSCAN         0.686267  0.471683       NaN   
                 CoHiRF-KernelRBF      0.429470  0.464621       NaN   
                 DBSCAN                0.717703  0.642741       NaN   
                 K-Means               0.608404  0.412353  0.412518   

                                           Calinski  Silhouette  \
Dataset          Model                                            
alizadeh-2000-v2 Batch CoHiRF-SC-SRGF     13.133888         NaN   
                 CoHiRF                   14.869219    0.189807   
                 CoHiRF-KernelRBF          1.183931    0.008144   
                 DBSCAN                   -1.000000   -1.000000   
                 K-Means                  15.150730    0.188241   
...                                             ...         ...   
shuttle          CoHiRF                16523.130402         NaN   
                 CoHiRF-DBSCAN          1055.982961         NaN   
                 CoHiRF-KernelRBF       5015.453473         NaN   
                 DBSCAN                 6602.153486         NaN   
                 K-Means               31177.656420    0.964295   

                                      Davies-Bouldin  Mean Best Time  \
Dataset          Model                                                 
alizadeh-2000-v2 Batch CoHiRF-SC-SRGF            NaN        3.592663   
                 CoHiRF                     2.828146        0.073616   
                 CoHiRF-KernelRBF           6.867684        0.329260   
                 DBSCAN                       1000.0        0.049808   
                 K-Means                     2.50609        0.022953   
...                                              ...             ...   
shuttle          CoHiRF                          NaN        4.559673   
                 CoHiRF-DBSCAN                   NaN       86.063949   
                 CoHiRF-KernelRBF                NaN        8.227903   
                 DBSCAN                          NaN       31.383453   
                 K-Means                    0.949712        0.155897   

                                       Mean HPO Time  
Dataset          Model                                
alizadeh-2000-v2 Batch CoHiRF-SC-SRGF     772.321511  
                 CoHiRF                   100.351318  
                 CoHiRF-KernelRBF          96.898627  
                 DBSCAN                    90.828893  
                 K-Means                   96.054563  
...                                              ...  
shuttle          CoHiRF                   307.049147  
                 CoHiRF-DBSCAN           5065.215894  
                 CoHiRF-KernelRBF         411.981844  
                 DBSCAN                  2072.228239  
                 K-Means                  248.079873  

[143 rows x 8 columns]

In [80]:
# Add mean time columns to the existing df_metrics dataframe
# Using the same filtering approach as the original df_metrics
df_filtered = df_runs_parents.loc[df_runs_parents['standardize'] == True].copy()
df_filtered = df_filtered.loc[df_filtered['model'].isin(model_names.keys())]
df_filtered = df_filtered.replace({"model": model_names})
df_filtered = df_filtered.replace({"dataset_name": dataset_names})

# Calculate mean times for each dataset-model combination across all metrics
df_times = df_filtered.groupby(['dataset_name', 'model']).agg({
    'best/elapsed_time': 'mean',
    'fit_model_return_elapsed_time': 'mean'
}).rename(columns={
    'best/elapsed_time': 'Mean Best Time',
    'fit_model_return_elapsed_time': 'Mean HPO Time'
})

# Set the same index structure as df_metrics
df_times.index.names = ["Dataset", "Model"]

# Join with the existing df_metrics (verify we have the same number of rows!)
df_metrics = df_metrics.join(df_times, how="outer")

In [82]:
# Create a time-based dataframe with elapsed times for each metric optimization
# Using the same filtering approach as the original df_metrics
df_filtered = df_runs_parents.loc[df_runs_parents['standardize'] == True].copy()
df_filtered = df_filtered.loc[df_filtered['model'].isin(model_names.keys())]
df_filtered = df_filtered.replace({"model": model_names})
df_filtered = df_filtered.replace({"dataset_name": dataset_names})

# Create separate dataframes for each metric optimization with time columns
df_ari_time = df_filtered.loc[df_filtered['hpo_metric'] == 'adjusted_rand'][
    ['dataset_name', 'model', 'best/elapsed_time', 'fit_model_return_elapsed_time']
].rename(columns={'best/elapsed_time': 'ARI_best_time', 'fit_model_return_elapsed_time': 'ARI_total_time'})

df_ami_time = df_filtered.loc[df_filtered['hpo_metric'] == 'adjusted_mutual_info'][
    ['dataset_name', 'model', 'best/elapsed_time', 'fit_model_return_elapsed_time']
].rename(columns={'best/elapsed_time': 'AMI_best_time', 'fit_model_return_elapsed_time': 'AMI_total_time'})

df_nmi_time = df_filtered.loc[df_filtered['hpo_metric'] == 'normalized_mutual_info'][
    ['dataset_name', 'model', 'best/elapsed_time', 'fit_model_return_elapsed_time']
].rename(columns={'best/elapsed_time': 'NMI_best_time', 'fit_model_return_elapsed_time': 'NMI_total_time'})

df_calinski_time = df_filtered.loc[df_filtered['hpo_metric'] == 'calinski_harabasz_score'][
    ['dataset_name', 'model', 'best/elapsed_time', 'fit_model_return_elapsed_time']
].rename(columns={'best/elapsed_time': 'Calinski_best_time', 'fit_model_return_elapsed_time': 'Calinski_total_time'})

df_silhouette_time = df_filtered.loc[df_filtered['hpo_metric'] == 'silhouette'][
    ['dataset_name', 'model', 'best/elapsed_time', 'fit_model_return_elapsed_time']
].rename(columns={'best/elapsed_time': 'Silhouette_best_time', 'fit_model_return_elapsed_time': 'Silhouette_total_time'})

df_davies_bouldin_time = df_filtered.loc[df_filtered['hpo_metric'] == 'davies_bouldin_score'][
    ['dataset_name', 'model', 'best/elapsed_time', 'fit_model_return_elapsed_time']
].rename(columns={'best/elapsed_time': 'Davies-Bouldin_best_time', 'fit_model_return_elapsed_time': 'Davies-Bouldin_total_time'})

# Remove missing values before setting index
df_ari_time = df_ari_time.dropna(subset=["ARI_best_time", "ARI_total_time"])
df_ami_time = df_ami_time.dropna(subset=["AMI_best_time", "AMI_total_time"])
df_nmi_time = df_nmi_time.dropna(subset=["NMI_best_time", "NMI_total_time"])
df_calinski_time = df_calinski_time.dropna(subset=["Calinski_best_time", "Calinski_total_time"])
df_silhouette_time = df_silhouette_time.dropna(subset=["Silhouette_best_time", "Silhouette_total_time"])
df_davies_bouldin_time = df_davies_bouldin_time.dropna(subset=["Davies-Bouldin_best_time", "Davies-Bouldin_total_time"])

# Set multi-index for all dataframes
df_ari_time = df_ari_time.set_index(["dataset_name", "model"])
df_ami_time = df_ami_time.set_index(["dataset_name", "model"])
df_nmi_time = df_nmi_time.set_index(["dataset_name", "model"])
df_calinski_time = df_calinski_time.set_index(["dataset_name", "model"])
df_silhouette_time = df_silhouette_time.set_index(["dataset_name", "model"])
df_davies_bouldin_time = df_davies_bouldin_time.set_index(["dataset_name", "model"])

# Combine all time metrics into a single dataframe using outer join
df_time_metrics = df_ari_time.join(df_ami_time, how="outer").join(df_nmi, how="outer").join(df_calinski_time, how="outer").join(df_silhouette_time, how="outer").join(df_davies_bouldin_time, how="outer")

# Rename index levels
df_time_metrics.index.names = ["Dataset", "Model"]

The following will provide the latex code for a clean table, we only need to make a little adjustement in the first line to delete the "key" and have only one header. For the longtable environment (full data) we need to add the "\*" at the end of lines we dont want to have a page break. We also should replace the entire begin{table} ... end{table} by begin{longtable} ... end{longtable} in the latex file.


In [84]:
df_latex = df_metrics.copy()
highlight_max_ari = partial(highlight_max, column_name="ARI")
highlight_max_ami = partial(highlight_max, column_name="AMI")
highlight_max_calinski = partial(highlight_max, column_name="Calinski")
underline_2nd_max_ari = partial(underline_2nd_max, column_name="ARI")
underline_2nd_max_ami = partial(underline_2nd_max, column_name="AMI")
underline_2nd_max_calinski = partial(underline_2nd_max, column_name="Calinski")
print(
    df_latex.style.apply(highlight_max_ari, subset="ARI", axis=None)
    .apply(underline_2nd_max_ari, subset="ARI", axis=None)
    .apply(highlight_max_ami, subset="AMI", axis=None)
    .apply(underline_2nd_max_ami, subset="AMI", axis=None)
    .apply(highlight_max_calinski, subset="Calinski", axis=None)
    .apply(underline_2nd_max_calinski, subset="Calinski", axis=None)
    .format(precision=3, na_rep="No Run", subset=["ARI", "AMI", "Calinski"])
    .format(formatter="{:4.3f}", subset=["Mean Best Time", "Mean HPO Time"])
    .to_latex(hrules=True, clines="skip-last;data", convert_css=True, column_format="llrrrrrrrr", environment="longtable")
)

\begin{longtable}{llrrrrrrrr}
\toprule
 &  & ARI & AMI & NMI & Calinski & Silhouette & Davies-Bouldin & Mean Best Time & Mean HPO Time \\
Dataset & Model &  &  &  &  &  &  &  &  \\
\midrule
\endfirsthead
\toprule
 &  & ARI & AMI & NMI & Calinski & Silhouette & Davies-Bouldin & Mean Best Time & Mean HPO Time \\
Dataset & Model &  &  &  &  &  &  &  &  \\
\midrule
\endhead
\midrule
\multicolumn{10}{r}{Continued on next page} \\
\midrule
\endfoot
\bottomrule
\endlastfoot
\multirow[c]{6}{*}{alizadeh-2000-v2} & Batch CoHiRF-SC-SRGF & 0.795 & 0.671 & nan & 13.134 & nan & nan & 3.593 & 772.322 \\
 & CoHiRF & \bfseries 0.865 & \bfseries 0.758 & 0.768753 & \underline{14.869} & 0.189807 & 2.828146 & 0.074 & 100.351 \\
 & CoHiRF-KernelRBF & 0.046 & 0.086 & 0.255128 & 1.184 & 0.008144 & 6.867684 & 0.329 & 96.899 \\
 & DBSCAN & 0.000 & 0.000 & 0.000000 & -1.000 & -1.000000 & 1000.000000 & 0.050 & 90.829 \\
 & K-Means & \underline{0.831} & \underline{0.751} & 0.756502 & \bfseries 15.151 & 0.188241 & 

In [ ]:
datasets_ids = [39, 61, 46779]
model_names = {
    "AffinityPropagation": "Affinity Propagation",
    "AverageAgglomerativeClustering": "Average Agglomerative",
    # "BatchCoHiRF": "Batch CoHiRF",
    # "BatchCoHiRF-1iter": "Batch CoHiRF-1iter",
    # "BatchCoHiRF-DBSCAN": "Batch CoHiRF-DBSCAN",
    # "BatchCoHiRF-DBSCAN-1iter": "Batch CoHiRF-DBSCAN-1iter",
    "CoHiRF-50": "CoHiRF",
    "CoHiRF-DBSCAN-50": "CoHiRF-DBSCAN",
    "CoHiRF-KernelRBF-50": "CoHiRF-KernelRBF",
    # "CoHiRF-100": "CoHiRF",
    # "CoHiRF-DBSCAN-100": "CoHiRF-DBSCAN",
    # "CoHiRF-KernelRBF-100": "CoHiRF-KernelRBF",
    "CompleteAgglomerativeClustering": "Complete Agglomerative",
    "DBSCAN": "DBSCAN",
    "HDBSCAN": "HDBSCAN",
    "IRFLLRR": "IRFLLRR",
    "KMeans": "K-Means",
    "MeanShift": "Mean Shift",
    "OPTICS": "OPTICS",
    "Proclus": "Proclus",
    "SingleAgglomerativeClustering": "Single Agglomerative",
    "SpectralClustering": "Spectral Clustering",
    "SpectralSubspaceRandomization": "SC-SRGF",
    "WardAgglomerativeClustering": "Ward's Method",
}
df = df_runs_parents.copy()
# get time of the child run
df = df.loc[df['dataset_id'].isin(datasets_ids)]
df = df.loc[df["model"].isin(model_names.keys())]
# df = df.loc[df['hpo_metric'] == 'adjusted_rand']
df = df.loc[df['standardize'] == True]
df['Parameters'] = df.apply(get_parameters_string, axis=1)
df = df[['model', 'dataset_name', 'best/adjusted_rand', 'Parameters', 'best/elapsed_time']]
df = df.replace({"model": model_names})
df = df.rename(columns={'best/adjusted_rand': 'ARI', 'model': 'Model', 'dataset_name': 'Dataset', 'best/elapsed_time': 'Time (s)'})
df = df.groupby(['Dataset', 'Model']).agg({'ARI': 'mean', 'Time (s)': 'first', 'Parameters': 'first'})

In [ ]:
df

ARI   Time (s)  \
Dataset     Model                                         
ecoli       Affinity Propagation    0.248155   0.053642   
            Average Agglomerative   0.727610   0.007877   
            CoHiRF                  0.756428   0.143971   
            CoHiRF-DBSCAN           0.454527   0.051831   
            CoHiRF-KernelRBF        0.704770   0.121266   
            Complete Agglomerative  0.785384   0.006143   
            DBSCAN                  0.573312   0.015961   
            HDBSCAN                 0.397572   0.019754   
            IRFLLRR                 0.452451   0.282542   
            K-Means                 0.723973   0.003528   
            Mean Shift              0.038054   2.280556   
            OPTICS                  0.313694   5.558529   
            Proclus                 0.429366   0.018805   
            SC-SRGF                 0.725076   2.121167   
            Single Agglomerative    0.063403   0.002310   
            Spectral Clustering     0.004373   0.191681   
            Ward's Method           0.734778   0.004258   
garber-2001 Affinity Propagation    0.075380   0.029222   
            Average Agglomerative   0.146050   0.027836   
            CoHiRF                  0.314324   0.170125   
            CoHiRF-KernelRBF        0.029670   2.884280   
            Complete Agglomerative  0.283306   0.033063   
            DBSCAN                  0.000000   0.055537   
            HDBSCAN                 0.026032   0.069505   
            IRFLLRR                 0.230480  14.448882   
            K-Means                 0.270265   0.031706   
            Mean Shift              0.016404   3.374457   
            OPTICS                  0.067677   0.182464   
            Proclus                 0.303213   9.575453   
            SC-SRGF                 0.142023   0.392039   
            Single Agglomerative    0.144825   0.027861   
            Spectral Clustering     0.138854   0.044788   
            Ward's Method           0.199979   0.026947   
iris        Affinity Propagation    0.477171   0.067877   
            Average Agglomerative   0.568116   0.001508   
            CoHiRF                  0.781855   0.046310   
            CoHiRF-DBSCAN           0.568116   0.046689   
            CoHiRF-KernelRBF        0.868111   0.065971   
            Complete Agglomerative  0.572631   0.002201   
            DBSCAN                  0.568116   0.017639   
            HDBSCAN                 0.563751   0.024827   
            IRFLLRR                 0.191720   0.109432   
            K-Means                 0.568116   0.002025   
            Mean Shift              0.568116   2.158059   
            OPTICS                  0.568116   2.480068   
            Proclus                 0.609526   0.033081   
            SC-SRGF                 0.766311   0.363099   
            Single Agglomerative    0.568116   0.001907   
            Spectral Clustering     0.568116   0.018037   
            Ward's Method           0.615323   0.001542   

                                                                           Parameters  
Dataset     Model                                                                      
ecoli       Affinity Propagation                                       $\lambda=0.58$  
            Average Agglomerative                                              $C=11$  
            CoHiRF                                             $C=4$; $q=0.99$; $R=6$  
            CoHiRF-DBSCAN           $\epsilon=0.43$; $n_{\text{min}}=13$; $q=0.42$...  
            CoHiRF-KernelRBF                    $C=2$; $q=0.27$; $R=9$; $\gamma=1.12$  
            Complete Agglomerative                                              $C=7$  
            DBSCAN                               $\epsilon=1.24$; $n_{\text{min}}=32$  
            HDBSCAN                                               $C_{\text{min}}=10$  
            IRFLLRR                 $\alpha=1$; $c=100$; $\lambda=0.10$; $p=0.88$;...  
    

In [ ]:
# with time, appendix
df_latex = df.copy()
highlight_max_ari = partial(highlight_max, column_name='ARI')
highlight_max_ari_index = partial(highlight_max_index, df_column=df_latex['ARI'])
underline_2nd_max_ari = partial(underline_2nd_max, column_name='ARI')
underline_2nd_max_ari_index = partial(underline_2nd_max_index, df_column=df_latex['ARI'])
print(df_latex.style.apply(highlight_max_ari, subset='ARI', axis=None).apply_index(highlight_max_ari_index,'index', level=1).apply(underline_2nd_max_ari, subset='ARI', axis=None).apply_index(underline_2nd_max_ari_index, 'index', level=1).format(precision=3, na_rep='No Run', subset='ARI').format(formatter='{:4.3f}',subset='Time (s)').to_latex(hrules=True, clines='skip-last;data', convert_css=True, column_format='llrrl'))

\begin{tabular}{llrrl}
\toprule
 &  & ARI & Time (s) & Parameters \\
Dataset & Model &  &  &  \\
\midrule
\multirow[c]{17}{*}{ecoli} & Affinity Propagation & 0.248 & 0.054 & $\lambda=0.58$ \\
 & Average Agglomerative & 0.728 & 0.008 & $C=11$ \\
 & \underline{CoHiRF} & \underline{0.756} & 0.144 & $C=4$; $q=0.99$; $R=6$ \\
 & CoHiRF-DBSCAN & 0.455 & 0.052 & $\epsilon=0.43$; $n_{\text{min}}=13$; $q=0.42$; $R=8$ \\
 & CoHiRF-KernelRBF & 0.705 & 0.121 & $C=2$; $q=0.27$; $R=9$; $\gamma=1.12$ \\
 & \bfseries Complete Agglomerative & \bfseries 0.785 & 0.006 & $C=7$ \\
 & DBSCAN & 0.573 & 0.016 & $\epsilon=1.24$; $n_{\text{min}}=32$ \\
 & HDBSCAN & 0.398 & 0.020 & $C_{\text{min}}=10$ \\
 & IRFLLRR & 0.452 & 0.283 & $\alpha=1$; $c=100$; $\lambda=0.10$; $p=0.88$; $C=8$ \\
 & K-Means & 0.724 & 0.004 & $C=6$ \\
 & Mean Shift & 0.038 & 2.281 & $bin_{\text{min}}=1$ \\
 & OPTICS & 0.314 & 5.559 & $n_{\text{min}}=10$ \\
 & Proclus & 0.429 & 0.019 & $d=0.58$; $C=2$ \\
 & SC-SRGF & 0.725 & 2.121 & $m=21$

In [ ]:
# no time, main text
df_latex = df.copy()[['ARI', 'Parameters']]
highlight_max_ari = partial(highlight_max, column_name='ARI')
highlight_max_ari_index = partial(highlight_max_index, df_column=df_latex['ARI'])
underline_2nd_max_ari = partial(underline_2nd_max, column_name='ARI')
underline_2nd_max_ari_index = partial(underline_2nd_max_index, df_column=df_latex['ARI'])
print(df_latex.style.apply(highlight_max_ari, subset='ARI', axis=None).apply_index(highlight_max_ari_index,'index', level=1).apply(underline_2nd_max_ari, subset='ARI', axis=None).apply_index(underline_2nd_max_ari_index, 'index', level=1).format(precision=3, na_rep='No Run', subset='ARI').to_latex(hrules=True, clines='skip-last;data', convert_css=True, column_format='p{0.95cm}lp{0.5cm}l'))

\begin{tabular}{llrrl}
\toprule
 & key & ARI & Parameters \\
Dataset & Model &  &  \\
\midrule
\multirow[c]{9}{*}{ecoli} & Affinity Propagation & 0.248 & $\lambda=0.58$ \\
 & \bfseries CoHiRF & \bfseries 0.758 & $C=7$; $q=11$; $R=10$ \\
 & \underline{CoHiRF-RBF} & \underline{0.742} & $C=7$; $q=25$; $R=4$ \\
 & DBSCAN & 0.345 & $n_{\text{min}}=7$; $\epsilon=0.78$ \\
 & HDBSCAN & 0.398 & $C_{\text{min}}=10$ \\
 & K-Means & 0.719 & $C=6$ \\
 & OPTICS & 0.314 & $n_{\text{min}}=10$ \\
 & SC-SRGF & 0.723 & $m=15$; $r=0.80$; $C=4$ \\
 & Ward's Method & 0.735 & $C=7$ \\
\cline{1-4}
\multirow[c]{10}{*}{har} & Affinity Propagation & 0.313 & $\lambda=1.00$ \\
 & CoHiRF & 0.491 & $C=4$; $q=11$; $R=8$ \\
 & CoHiRF-1000 & 0.341 & $C=4$; $q=18$; $R=3$ \\
 & CoHiRF-RBF & 0.495 & $C=6$; $q=13$; $R=4$ \\
 & DBSCAN & 0.302 & $n_{\text{min}}=3$; $\epsilon=13.91$ \\
 & HDBSCAN & 0.287 & $C_{\text{min}}=6$ \\
 & K-Means & 0.438 & $C=9$ \\
 & OPTICS & 0.001 & $n_{\text{min}}=4$ \\
 & \bfseries SC-SRGF & \bfs

# Debug and explore

In [ ]:
df = df_runs_raw_parents.copy()

In [ ]:
model_names = df.model.unique().tolist()
model_names.sort()
model_names

['AffinityPropagation',
 'AverageAgglomerativeClustering',
 'BatchCoHiRF-1iter',
 'BatchCoHiRF-DBSCAN-1iter',
 'BatchCoHiRF-SC-SRGF',
 'CoHiRF',
 'CoHiRF-DBSCAN',
 'CoHiRF-KernelRBF',
 'CompleteAgglomerativeClustering',
 'DBSCAN',
 'HDBSCAN',
 'IRFLLRR',
 'KMeans',
 'MeanShift',
 'OPTICS',
 'Proclus',
 'SingleAgglomerativeClustering',
 'SpectralClustering',
 'SpectralSubspaceRandomization',
 'WardAgglomerativeClustering']

In [ ]:
model_names = [
    "AffinityPropagation",
    # "AverageAgglomerativeClustering",
    # "BatchCoHiRF-1iter",
    # "BatchCoHiRF-DBSCAN-1iter",
    # "BatchCoHiRF-SC-SRGF",
    # "CoHiRF",
    # "CoHiRF-DBSCAN",
    # "CoHiRF-KernelRBF",
    # "CompleteAgglomerativeClustering",
    # "DBSCAN",
    # "HDBSCAN",
    # "IRFLLRR",
    # "KMeans",
    # "MeanShift",
    # "OPTICS",
    # "Proclus",
    # "SingleAgglomerativeClustering",
    # "SpectralClustering",
    "SpectralSubspaceRandomization",
    # "WardAgglomerativeClustering",
]

In [ ]:
df = df.loc[df["best/adjusted_rand"].isna()]
df = df.loc[~df["model"].isin(model_names)]

In [ ]:
df

,status,start_time,end_time,best/alpha,best/avg_dims,best/base_model_kwargs/eps,best/base_model_kwargs/min_samples,best/base_model_kwargs/n_clusters,best/c,best/child_run_id,...,max_memory_used,max_memory_used_after_fit,mlflow.parentRunId,raised_exception,dataset,openml_id,n_instances,n_features,n_classes,n_categorical
run_uuid,,,,,,,,,,,,,,,,,,,,,
0901bfc08af5458ab0034ab7cb7d1b2e,FINISHED,1751916368453,1.751917e+12,NaN,NaN,NaN,NaN,NaN,NaN,820cc0feddd54385b382eb25ecca62f2,...,493.620,493.620,NaN,False,bittner-2000,46776,38,2202,2,0
1d574921f2c543b78cee887c28f3946b,FINISHED,1751915421602,1.751916e+12,NaN,NaN,NaN,NaN,NaN,NaN,2d010d3203d94fa986be77f749c84e2e,...,482.804,482.804,NaN,False,golub-1999-v2,46780,72,1869,3,0
33cb421921c041b584362f17a4cdc17d,FINISHED,1751914443733,1.751915e+12,NaN,NaN,NaN,NaN,NaN,NaN,731020e814df4d46a5ff2bd6984e121d,...,467.028,467.028,NaN,False,bredel-2005,46777,50,1740,3,0
3fbaefa4ed844048ae4989264777d062,FINISHED,1751914724869,1.751915e+12,NaN,NaN,0.5,5.0,NaN,NaN,1b2f91a4346b4e3b83e2c881da933fe0,...,436.040,436.040,NaN,False,khan-2001,46781,83,1070,4,0
6779db1b4fe343a99e9fb55206dfedb0,FINISHED,1751915142539,1.751915e+12,NaN,NaN,NaN,NaN,NaN,NaN,138f534b142d455c99599b996f185fa3,...,440.360,440.360,NaN,False,khan-2001,46781,83,1070,4,0
695d4e9d098f4dc69268794dc6b7b2d6,FINISHED,1751915044106,1.751915e+12,NaN,NaN,NaN,NaN,NaN,NaN,b250df6764e44fd48dc6d7ca77495729,...,491.192,491.192,NaN,False,alizadeh-2000-v3,46774,62,2092,4,0
6afd8f179fc54e86b856e958b268e4cd,FINISHED,1751914203808,1.751915e+12,4.0,NaN,NaN,NaN,NaN,0.1,eccef9f294b14c35a5a3bae860af6d25,...,30077.172,30077.172,NaN,False,shuttle,40685,58000,10,7,1
6e6a7953d76542ea8dbabdb462ee2218,FINISHED,1751916127683,1.751916e+12,NaN,NaN,NaN,NaN,NaN,NaN,fca5ad03fa2846508b7d57905ae855e1,...,495.456,495.456,NaN,False,alizadeh-2000-v3,46774,62,2092,4,0
76965c65b70f43ae855292070c4ca909,FINISHED,1751914487224,1.751915e+12,NaN,NaN,0.5,5.0,NaN,NaN,2799568ae82248748739957d0cce4efe,...,470.040,470.040,NaN,False,bredel-2005,46777,50,1740,3,0


In [ ]:
runs_to_delete_parents = list(df.index)

In [ ]:
df = df_runs_raw.copy()
df = df.loc[df["mlflow.parentRunId"].isin(runs_to_delete_parents)]

In [ ]:
runs_to_delete_children = list(df.index)

In [ ]:
runs_to_delete = runs_to_delete_children + runs_to_delete_parents

In [ ]:
run_uuid_query = [f"'{run_id}'" for run_id in runs_to_delete]
run_uuid_query = ', '.join(run_uuid_query)

In [ ]:
query = f"""
UPDATE runs
SET lifecycle_stage = 'deleted'
WHERE run_uuid IN ({run_uuid_query}) 
"""
with engine.begin() as conn:
    conn.execute(text(query))

In [ ]:
query = f"""
DELETE
FROM
	experiment_tags
WHERE
	experiment_id = ANY(
	SELECT
		experiment_id
	FROM
		experiments
	WHERE
		lifecycle_stage = 'deleted');

DELETE
FROM
	latest_metrics
WHERE
	run_uuid = ANY(
	SELECT
		run_uuid
	FROM
		runs
	WHERE
		lifecycle_stage = 'deleted');
	
DELETE
FROM
	metrics
WHERE
	run_uuid = ANY(
	SELECT
		run_uuid
	FROM
		runs
	WHERE
		lifecycle_stage = 'deleted');
	
DELETE
FROM
	params
WHERE
	run_uuid = ANY(
	SELECT
		run_uuid
	FROM
		runs
	WHERE
		lifecycle_stage = 'deleted');

DELETE
FROM
	tags
WHERE
	run_uuid = ANY(
	SELECT
		run_uuid
	FROM
		runs
	WHERE
		lifecycle_stage = 'deleted');
	
DELETE 
FROM 
	runs
WHERE 
	lifecycle_stage = 'deleted';

DELETE 
FROM 
	experiments
WHERE 
	lifecycle_stage = 'deleted';
"""
with engine.begin() as conn:
    conn.execute(text(query))

for i, row in df_runs_raw.iterrows():
    run_id = row.run_id
    model_name = row['params.model_name']
    with mlflow.start_run(run_id) as run:
        mlflow.log_param('model_nickname', model_name)    

In [ ]:
# Display summary information about the metrics dataframe
print("Dataframe shape:", df_metrics.shape)
print("\nColumns:", df_metrics.columns.tolist())
print("\nIndex levels:", df_metrics.index.names)
print("\nFirst few rows:")
print(df_metrics.head(10))
print("\nData types:")
print(df_metrics.dtypes)
print("\nNon-null counts:")
print(df_metrics.count())

Dataframe shape: (335, 3)

Columns: ['ARI', 'AMI', 'Calinski']

Index levels: ['Dataset', 'Model']

First few rows:
                                                       ARI       AMI  \
Dataset          Model                                                 
alizadeh-2000-v2 AffinityPropagation              0.362816       NaN   
                 AverageAgglomerativeClustering   0.809591  0.676228   
                 BatchCoHiRF-SC-SRGF              0.794763  0.671212   
                 CoHiRF                           0.864606  0.757990   
                 CoHiRF-KernelRBF                 0.046212  0.085768   
                 CompleteAgglomerativeClustering  0.382805  0.612835   
                 DBSCAN                           0.000000  0.000000   
                 HDBSCAN                          0.165928  0.267066   
                 IRFLLRR                          0.512283       NaN   
                 KMeans                           0.830673  0.750678   

                   

The following will provide the latex code for a clean table, we only need to make a little adjustement in the first line to delete the "key" and have only one header. For the longtable environment (full data) we need to add the "\*" at the end of lines we dont want to have a page break. We also should replace the entire begin{table} ... end{table} by begin{longtable} ... end{longtable} in the latex file.


In [43]:
datasets_ids = [39, 61, 46779]
model_names = {
    "AffinityPropagation": "Affinity Propagation",
    "AverageAgglomerativeClustering": "Average Agglomerative",
    # "BatchCoHiRF": "Batch CoHiRF",
    # "BatchCoHiRF-1iter": "Batch CoHiRF-1iter",
    # "BatchCoHiRF-DBSCAN": "Batch CoHiRF-DBSCAN",
    # "BatchCoHiRF-DBSCAN-1iter": "Batch CoHiRF-DBSCAN-1iter",
    "CoHiRF-50": "CoHiRF",
    "CoHiRF-DBSCAN-50": "CoHiRF-DBSCAN",
    "CoHiRF-KernelRBF-50": "CoHiRF-KernelRBF",
    # "CoHiRF-100": "CoHiRF",
    # "CoHiRF-DBSCAN-100": "CoHiRF-DBSCAN",
    # "CoHiRF-KernelRBF-100": "CoHiRF-KernelRBF",
    "CompleteAgglomerativeClustering": "Complete Agglomerative",
    "DBSCAN": "DBSCAN",
    "HDBSCAN": "HDBSCAN",
    "IRFLLRR": "IRFLLRR",
    "KMeans": "K-Means",
    "MeanShift": "Mean Shift",
    "OPTICS": "OPTICS",
    "Proclus": "Proclus",
    "SingleAgglomerativeClustering": "Single Agglomerative",
    "SpectralClustering": "Spectral Clustering",
    "SpectralSubspaceRandomization": "SC-SRGF",
    "WardAgglomerativeClustering": "Ward's Method",
}
df = df_runs_parents.copy()
# get time of the child run
df = df.loc[df['dataset_id'].isin(datasets_ids)]
df = df.loc[df["model"].isin(model_names.keys())]
# df = df.loc[df['hpo_metric'] == 'adjusted_rand']
df = df.loc[df['standardize'] == True]
df['Parameters'] = df.apply(get_parameters_string, axis=1)
df = df[['model', 'dataset_name', 'best/adjusted_rand', 'Parameters', 'best/elapsed_time']]
df = df.replace({"model": model_names})
df = df.rename(columns={'best/adjusted_rand': 'ARI', 'model': 'Model', 'dataset_name': 'Dataset', 'best/elapsed_time': 'Time (s)'})
df = df.groupby(['Dataset', 'Model']).agg({'ARI': 'mean', 'Time (s)': 'first', 'Parameters': 'first'})

In [44]:
df

ARI   Time (s)  \
Dataset     Model                                         
ecoli       Affinity Propagation    0.248155   0.053642   
            Average Agglomerative   0.727610   0.007877   
            CoHiRF                  0.756428   0.143971   
            CoHiRF-DBSCAN           0.454527   0.051831   
            CoHiRF-KernelRBF        0.704770   0.121266   
            Complete Agglomerative  0.785384   0.006143   
            DBSCAN                  0.573312   0.015961   
            HDBSCAN                 0.397572   0.019754   
            IRFLLRR                 0.452451   0.282542   
            K-Means                 0.723973   0.003528   
            Mean Shift              0.038054   2.280556   
            OPTICS                  0.313694   5.558529   
            Proclus                 0.429366   0.018805   
            SC-SRGF                 0.725076   2.121167   
            Single Agglomerative    0.063403   0.002310   
            Spectral Clustering     0.004373   0.191681   
            Ward's Method           0.734778   0.004258   
garber-2001 Affinity Propagation    0.075380   0.029222   
            Average Agglomerative   0.146050   0.027836   
            CoHiRF                  0.314324   0.170125   
            CoHiRF-KernelRBF        0.029670   2.884280   
            Complete Agglomerative  0.283306   0.033063   
            DBSCAN                  0.000000   0.055537   
            HDBSCAN                 0.026032   0.069505   
            IRFLLRR                 0.230480  14.448882   
            K-Means                 0.270265   0.031706   
            Mean Shift              0.016404   3.374457   
            OPTICS                  0.067677   0.182464   
            Proclus                 0.303213   9.575453   
            SC-SRGF                 0.142023   0.392039   
            Single Agglomerative    0.144825   0.027861   
            Spectral Clustering     0.138854   0.044788   
            Ward's Method           0.199979   0.026947   
iris        Affinity Propagation    0.477171   0.067877   
            Average Agglomerative   0.568116   0.001508   
            CoHiRF                  0.781855   0.046310   
            CoHiRF-DBSCAN           0.568116   0.046689   
            CoHiRF-KernelRBF        0.868111   0.065971   
            Complete Agglomerative  0.572631   0.002201   
            DBSCAN                  0.568116   0.017639   
            HDBSCAN                 0.563751   0.024827   
            IRFLLRR                 0.191720   0.109432   
            K-Means                 0.568116   0.002025   
            Mean Shift              0.568116   2.158059   
            OPTICS                  0.568116   2.480068   
            Proclus                 0.609526   0.033081   
            SC-SRGF                 0.766311   0.363099   
            Single Agglomerative    0.568116   0.001907   
            Spectral Clustering     0.568116   0.018037   
            Ward's Method           0.615323   0.001542   

                                                                           Parameters  
Dataset     Model                                                                      
ecoli       Affinity Propagation                                       $\lambda=0.58$  
            Average Agglomerative                                              $C=11$  
            CoHiRF                                             $C=4$; $q=0.99$; $R=6$  
            CoHiRF-DBSCAN           $\epsilon=0.43$; $n_{\text{min}}=13$; $q=0.42$...  
            CoHiRF-KernelRBF                    $C=2$; $q=0.27$; $R=9$; $\gamma=1.12$  
            Complete Agglomerative                                              $C=7$  
            DBSCAN                               $\epsilon=1.24$; $n_{\text{min}}=32$  
            HDBSCAN                                               $C_{\text{min}}=10$  
            IRFLLRR                 $\alpha=1$; $c=100$; $\lambda=0.10$; $p=0.88$;...  
    

In [45]:
# with time, appendix
df_latex = df.copy()
highlight_max_ari = partial(highlight_max, column_name='ARI')
highlight_max_ari_index = partial(highlight_max_index, df_column=df_latex['ARI'])
underline_2nd_max_ari = partial(underline_2nd_max, column_name='ARI')
underline_2nd_max_ari_index = partial(underline_2nd_max_index, df_column=df_latex['ARI'])
print(df_latex.style.apply(highlight_max_ari, subset='ARI', axis=None).apply_index(highlight_max_ari_index,'index', level=1).apply(underline_2nd_max_ari, subset='ARI', axis=None).apply_index(underline_2nd_max_ari_index, 'index', level=1).format(precision=3, na_rep='No Run', subset='ARI').format(formatter='{:4.3f}',subset='Time (s)').to_latex(hrules=True, clines='skip-last;data', convert_css=True, column_format='llrrl'))

\begin{tabular}{llrrl}
\toprule
 &  & ARI & Time (s) & Parameters \\
Dataset & Model &  &  &  \\
\midrule
\multirow[c]{17}{*}{ecoli} & Affinity Propagation & 0.248 & 0.054 & $\lambda=0.58$ \\
 & Average Agglomerative & 0.728 & 0.008 & $C=11$ \\
 & \underline{CoHiRF} & \underline{0.756} & 0.144 & $C=4$; $q=0.99$; $R=6$ \\
 & CoHiRF-DBSCAN & 0.455 & 0.052 & $\epsilon=0.43$; $n_{\text{min}}=13$; $q=0.42$; $R=8$ \\
 & CoHiRF-KernelRBF & 0.705 & 0.121 & $C=2$; $q=0.27$; $R=9$; $\gamma=1.12$ \\
 & \bfseries Complete Agglomerative & \bfseries 0.785 & 0.006 & $C=7$ \\
 & DBSCAN & 0.573 & 0.016 & $\epsilon=1.24$; $n_{\text{min}}=32$ \\
 & HDBSCAN & 0.398 & 0.020 & $C_{\text{min}}=10$ \\
 & IRFLLRR & 0.452 & 0.283 & $\alpha=1$; $c=100$; $\lambda=0.10$; $p=0.88$; $C=8$ \\
 & K-Means & 0.724 & 0.004 & $C=6$ \\
 & Mean Shift & 0.038 & 2.281 & $bin_{\text{min}}=1$ \\
 & OPTICS & 0.314 & 5.559 & $n_{\text{min}}=10$ \\
 & Proclus & 0.429 & 0.019 & $d=0.58$; $C=2$ \\
 & SC-SRGF & 0.725 & 2.121 & $m=21$

In [ ]:
# no time, main text
df_latex = df.copy()[['ARI', 'Parameters']]
highlight_max_ari = partial(highlight_max, column_name='ARI')
highlight_max_ari_index = partial(highlight_max_index, df_column=df_latex['ARI'])
underline_2nd_max_ari = partial(underline_2nd_max, column_name='ARI')
underline_2nd_max_ari_index = partial(underline_2nd_max_index, df_column=df_latex['ARI'])
print(df_latex.style.apply(highlight_max_ari, subset='ARI', axis=None).apply_index(highlight_max_ari_index,'index', level=1).apply(underline_2nd_max_ari, subset='ARI', axis=None).apply_index(underline_2nd_max_ari_index, 'index', level=1).format(precision=3, na_rep='No Run', subset='ARI').to_latex(hrules=True, clines='skip-last;data', convert_css=True, column_format='p{0.95cm}lp{0.5cm}l'))

\begin{tabular}{llrrl}
\toprule
 & key & ARI & Parameters \\
Dataset & Model &  &  \\
\midrule
\multirow[c]{9}{*}{ecoli} & Affinity Propagation & 0.248 & $\lambda=0.58$ \\
 & \bfseries CoHiRF & \bfseries 0.758 & $C=7$; $q=11$; $R=10$ \\
 & \underline{CoHiRF-RBF} & \underline{0.742} & $C=7$; $q=25$; $R=4$ \\
 & DBSCAN & 0.345 & $n_{\text{min}}=7$; $\epsilon=0.78$ \\
 & HDBSCAN & 0.398 & $C_{\text{min}}=10$ \\
 & K-Means & 0.719 & $C=6$ \\
 & OPTICS & 0.314 & $n_{\text{min}}=10$ \\
 & SC-SRGF & 0.723 & $m=15$; $r=0.80$; $C=4$ \\
 & Ward's Method & 0.735 & $C=7$ \\
\cline{1-4}
\multirow[c]{10}{*}{har} & Affinity Propagation & 0.313 & $\lambda=1.00$ \\
 & CoHiRF & 0.491 & $C=4$; $q=11$; $R=8$ \\
 & CoHiRF-1000 & 0.341 & $C=4$; $q=18$; $R=3$ \\
 & CoHiRF-RBF & 0.495 & $C=6$; $q=13$; $R=4$ \\
 & DBSCAN & 0.302 & $n_{\text{min}}=3$; $\epsilon=13.91$ \\
 & HDBSCAN & 0.287 & $C_{\text{min}}=6$ \\
 & K-Means & 0.438 & $C=9$ \\
 & OPTICS & 0.001 & $n_{\text{min}}=4$ \\
 & \bfseries SC-SRGF & \bfs

# Debug and explore

In [142]:
df = df_runs_raw_parents.copy()

In [143]:
model_names = df.model.unique().tolist()
model_names.sort()
model_names

['AffinityPropagation',
 'AverageAgglomerativeClustering',
 'BatchCoHiRF-1iter',
 'BatchCoHiRF-DBSCAN-1iter',
 'BatchCoHiRF-SC-SRGF',
 'CoHiRF',
 'CoHiRF-DBSCAN',
 'CoHiRF-KernelRBF',
 'CompleteAgglomerativeClustering',
 'DBSCAN',
 'HDBSCAN',
 'IRFLLRR',
 'KMeans',
 'MeanShift',
 'OPTICS',
 'Proclus',
 'SingleAgglomerativeClustering',
 'SpectralClustering',
 'SpectralSubspaceRandomization',
 'WardAgglomerativeClustering']

In [144]:
model_names = [
    "AffinityPropagation",
    # "AverageAgglomerativeClustering",
    # "BatchCoHiRF-1iter",
    # "BatchCoHiRF-DBSCAN-1iter",
    # "BatchCoHiRF-SC-SRGF",
    # "CoHiRF",
    # "CoHiRF-DBSCAN",
    # "CoHiRF-KernelRBF",
    # "CompleteAgglomerativeClustering",
    # "DBSCAN",
    # "HDBSCAN",
    # "IRFLLRR",
    # "KMeans",
    # "MeanShift",
    # "OPTICS",
    # "Proclus",
    # "SingleAgglomerativeClustering",
    # "SpectralClustering",
    "SpectralSubspaceRandomization",
    # "WardAgglomerativeClustering",
]

In [145]:
df = df.loc[df["best/adjusted_rand"].isna()]
df = df.loc[~df["model"].isin(model_names)]

In [146]:
df

,status,start_time,end_time,best/alpha,best/avg_dims,best/base_model_kwargs/eps,best/base_model_kwargs/min_samples,best/base_model_kwargs/n_clusters,best/c,best/child_run_id,...,max_memory_used,max_memory_used_after_fit,mlflow.parentRunId,raised_exception,dataset,openml_id,n_instances,n_features,n_classes,n_categorical
run_uuid,,,,,,,,,,,,,,,,,,,,,
0901bfc08af5458ab0034ab7cb7d1b2e,FINISHED,1751916368453,1.751917e+12,NaN,NaN,NaN,NaN,NaN,NaN,820cc0feddd54385b382eb25ecca62f2,...,493.620,493.620,NaN,False,bittner-2000,46776,38,2202,2,0
1d574921f2c543b78cee887c28f3946b,FINISHED,1751915421602,1.751916e+12,NaN,NaN,NaN,NaN,NaN,NaN,2d010d3203d94fa986be77f749c84e2e,...,482.804,482.804,NaN,False,golub-1999-v2,46780,72,1869,3,0
33cb421921c041b584362f17a4cdc17d,FINISHED,1751914443733,1.751915e+12,NaN,NaN,NaN,NaN,NaN,NaN,731020e814df4d46a5ff2bd6984e121d,...,467.028,467.028,NaN,False,bredel-2005,46777,50,1740,3,0
3fbaefa4ed844048ae4989264777d062,FINISHED,1751914724869,1.751915e+12,NaN,NaN,0.5,5.0,NaN,NaN,1b2f91a4346b4e3b83e2c881da933fe0,...,436.040,436.040,NaN,False,khan-2001,46781,83,1070,4,0
6779db1b4fe343a99e9fb55206dfedb0,FINISHED,1751915142539,1.751915e+12,NaN,NaN,NaN,NaN,NaN,NaN,138f534b142d455c99599b996f185fa3,...,440.360,440.360,NaN,False,khan-2001,46781,83,1070,4,0
695d4e9d098f4dc69268794dc6b7b2d6,FINISHED,1751915044106,1.751915e+12,NaN,NaN,NaN,NaN,NaN,NaN,b250df6764e44fd48dc6d7ca77495729,...,491.192,491.192,NaN,False,alizadeh-2000-v3,46774,62,2092,4,0
6afd8f179fc54e86b856e958b268e4cd,FINISHED,1751914203808,1.751915e+12,4.0,NaN,NaN,NaN,NaN,0.1,eccef9f294b14c35a5a3bae860af6d25,...,30077.172,30077.172,NaN,False,shuttle,40685,58000,10,7,1
6e6a7953d76542ea8dbabdb462ee2218,FINISHED,1751916127683,1.751916e+12,NaN,NaN,NaN,NaN,NaN,NaN,fca5ad03fa2846508b7d57905ae855e1,...,495.456,495.456,NaN,False,alizadeh-2000-v3,46774,62,2092,4,0
76965c65b70f43ae855292070c4ca909,FINISHED,1751914487224,1.751915e+12,NaN,NaN,0.5,5.0,NaN,NaN,2799568ae82248748739957d0cce4efe,...,470.040,470.040,NaN,False,bredel-2005,46777,50,1740,3,0


In [147]:
runs_to_delete_parents = list(df.index)

In [148]:
df = df_runs_raw.copy()
df = df.loc[df["mlflow.parentRunId"].isin(runs_to_delete_parents)]

In [149]:
runs_to_delete_children = list(df.index)

In [150]:
runs_to_delete = runs_to_delete_children + runs_to_delete_parents

In [151]:
run_uuid_query = [f"'{run_id}'" for run_id in runs_to_delete]
run_uuid_query = ', '.join(run_uuid_query)

In [152]:
query = f"""
UPDATE runs
SET lifecycle_stage = 'deleted'
WHERE run_uuid IN ({run_uuid_query}) 
"""
with engine.begin() as conn:
    conn.execute(text(query))

In [153]:
query = f"""
DELETE
FROM
	experiment_tags
WHERE
	experiment_id = ANY(
	SELECT
		experiment_id
	FROM
		experiments
	WHERE
		lifecycle_stage = 'deleted');

DELETE
FROM
	latest_metrics
WHERE
	run_uuid = ANY(
	SELECT
		run_uuid
	FROM
		runs
	WHERE
		lifecycle_stage = 'deleted');
	
DELETE
FROM
	metrics
WHERE
	run_uuid = ANY(
	SELECT
		run_uuid
	FROM
		runs
	WHERE
		lifecycle_stage = 'deleted');
	
DELETE
FROM
	params
WHERE
	run_uuid = ANY(
	SELECT
		run_uuid
	FROM
		runs
	WHERE
		lifecycle_stage = 'deleted');

DELETE
FROM
	tags
WHERE
	run_uuid = ANY(
	SELECT
		run_uuid
	FROM
		runs
	WHERE
		lifecycle_stage = 'deleted');
	
DELETE 
FROM 
	runs
WHERE 
	lifecycle_stage = 'deleted';

DELETE 
FROM 
	experiments
WHERE 
	lifecycle_stage = 'deleted';
"""
with engine.begin() as conn:
    conn.execute(text(query))

for i, row in df_runs_raw.iterrows():
    run_id = row.run_id
    model_name = row['params.model_name']
    with mlflow.start_run(run_id) as run:
        mlflow.log_param('model_nickname', model_name)    

In [40]:
# Display summary information about the metrics dataframe
print("Dataframe shape:", df_metrics.shape)
print("\nColumns:", df_metrics.columns.tolist())
print("\nIndex levels:", df_metrics.index.names)
print("\nFirst few rows:")
print(df_metrics.head(10))
print("\nData types:")
print(df_metrics.dtypes)
print("\nNon-null counts:")
print(df_metrics.count())

Dataframe shape: (335, 3)

Columns: ['ARI', 'AMI', 'Calinski']

Index levels: ['Dataset', 'Model']

First few rows:
                                                       ARI       AMI  \
Dataset          Model                                                 
alizadeh-2000-v2 AffinityPropagation              0.362816       NaN   
                 AverageAgglomerativeClustering   0.809591  0.676228   
                 BatchCoHiRF-SC-SRGF              0.794763  0.671212   
                 CoHiRF                           0.864606  0.757990   
                 CoHiRF-KernelRBF                 0.046212  0.085768   
                 CompleteAgglomerativeClustering  0.382805  0.612835   
                 DBSCAN                           0.000000  0.000000   
                 HDBSCAN                          0.165928  0.267066   
                 IRFLLRR                          0.512283       NaN   
                 KMeans                           0.830673  0.750678   

                   